In [1]:
!pip install keras-tcn
!pip install -U finance-datareader

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable


You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [2]:
import numpy as np
import pandas as pd
import FinanceDataReader as fdr
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tcn import TCN

/home/work/.local/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.0' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
2023-01-25 02:24:46.556309: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-25 02:24:46.712271: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-01-25 02:24:50.173357: W tensorflow/compiler/xla/stream_executor/platform/default/

In [3]:
kospi_train = fdr.DataReader('KS11', '1990-01-01','2017-12-31')['Close']
print(kospi_train.head())
print(kospi_train)
print(kospi_train.shape)
print(kospi_train.isnull().any())

Date
1996-12-11    704.679993
1996-12-12    689.380005
1996-12-13    689.070007
1996-12-16    673.919983
1996-12-17    663.349976
Name: Close, dtype: float64
Date
1996-12-11     704.679993
1996-12-12     689.380005
1996-12-13     689.070007
1996-12-16     673.919983
1996-12-17     663.349976
                 ...     
2017-12-21    2429.830078
2017-12-22    2440.540039
2017-12-26    2427.340088
2017-12-27    2436.669922
2017-12-28    2467.489990
Name: Close, Length: 5341, dtype: float64
(5341,)
True


In [4]:
kospi_test = fdr.DataReader('KS11', '2018-01-01')['Close']
print(kospi_test)
print(kospi_test.shape)
print(kospi_test.isnull().any())

Date
2018-01-02    2479.649902
2018-01-03    2486.350098
2018-01-04    2466.459961
2018-01-05    2497.520020
2018-01-08    2513.280029
                 ...     
2023-01-17    2379.389893
2023-01-18    2368.320068
2023-01-19    2380.340088
2023-01-20    2395.260010
2023-01-25    2428.360107
Name: Close, Length: 1246, dtype: float64
(1246,)
False


In [5]:
kospi_train = kospi_train.dropna()
kospi_test = kospi_test.dropna()

print(kospi_train)
print(kospi_train.isnull().any())
print(kospi_test)
print(kospi_test.isnull().any())

Date
1996-12-11     704.679993
1996-12-12     689.380005
1996-12-13     689.070007
1996-12-16     673.919983
1996-12-17     663.349976
                 ...     
2017-12-21    2429.830078
2017-12-22    2440.540039
2017-12-26    2427.340088
2017-12-27    2436.669922
2017-12-28    2467.489990
Name: Close, Length: 5192, dtype: float64
False
Date
2018-01-02    2479.649902
2018-01-03    2486.350098
2018-01-04    2466.459961
2018-01-05    2497.520020
2018-01-08    2513.280029
                 ...     
2023-01-17    2379.389893
2023-01-18    2368.320068
2023-01-19    2380.340088
2023-01-20    2395.260010
2023-01-25    2428.360107
Name: Close, Length: 1246, dtype: float64
False


In [6]:
kospi_train = kospi_train.values
kospi_test = kospi_test.values

In [31]:
lookback_window = 60
x_train_1, y_train_1 = [], []
x_train_7, y_train_7 = [], []
x_train_15, y_train_15 = [], []
x_train_30, y_train_30 = [], []

In [32]:
for i in range(lookback_window, len(kospi_train)-1):
    x_train_1.append(kospi_train[i - lookback_window:i])
    y_train_1.append(kospi_train[i] - kospi_train[i-1])

for i in range(lookback_window, len(kospi_train)-6):
    x_train_7.append(kospi_train[i - lookback_window:i])
    y_train_7.append(kospi_train[i+6] - kospi_train[i-1])

for i in range(lookback_window, len(kospi_train)-14):
    x_train_15.append(kospi_train[i - lookback_window:i])
    y_train_15.append(kospi_train[i+14] - kospi_train[i-1])

for i in range(lookback_window, len(kospi_train)-29):
    x_train_30.append(kospi_train[i - lookback_window:i])
    y_train_30.append(kospi_train[i+29] - kospi_train[i-1])

In [33]:
for index in range(len(y_train_1)):
    if y_train_1[index]>= -0.5 and y_train_1[index] < 0.5:
        y_train_1[index] = 'neutral'
    elif y_train_1[index] >= 0.5:
        y_train_1[index] = 'increase'
    else:
        y_train_1[index] = 'decrease'
        
for index in range(len(y_train_7)):
    if y_train_7[index]>= -0.5 and y_train_7[index] < 0.5:
        y_train_7[index] = 'neutral'
    elif y_train_7[index] >= 0.5:
        y_train_7[index] = 'increase'
    else:
        y_train_7[index] = 'decrease'

for index in range(len(y_train_15)):
    if y_train_15[index]>= -0.5 and y_train_15[index] < 0.5:
        y_train_15[index] = 'neutral'
    elif y_train_15[index] >= 0.5:
        y_train_15[index] = 'increase'
    else:
        y_train_15[index] = 'decrease'

for index in range(len(y_train_30)):
    if y_train_30[index]>= -0.5 and y_train_30[index] < 0.5:
        y_train_30[index] = 'neutral'
    elif y_train_30[index] >= 0.5:
        y_train_30[index] = 'increase'
    else:
        y_train_30[index] = 'decrease'

In [34]:
x_train_1 = np.array(x_train_1)
y_train_1 = np.array(y_train_1)
x_train_1 = x_train_1.reshape((5131, -1, 1))
#y_train_1 = y_train_1.reshape((-1, 1))

x_train_7 = np.array(x_train_7)
y_train_7 = np.array(y_train_7)
x_train_7 = x_train_7.reshape((5126, -1, 1))
#y_train_7 = y_train_7.reshape((-1, 1))

x_train_15 = np.array(x_train_15)
y_train_15 = np.array(y_train_15)
x_train_15 = x_train_15.reshape((5118, -1, 1))
#y_train_15 = y_train_15.reshape((-1, 1))

x_train_30 = np.array(x_train_30)
y_train_30 = np.array(y_train_30)
x_train_30 = x_train_30.reshape((5103, -1, 1))
#y_train_30 = y_train_30.reshape((-1, 1))

print(x_train_1.shape, x_train_7.shape, x_train_15.shape, x_train_30.shape)
print(y_train_1.shape, y_train_7.shape, y_train_15.shape, y_train_30.shape)

(5131, 60, 1) (5126, 60, 1) (5118, 60, 1) (5103, 60, 1)
(5131,) (5126,) (5118,) (5103,)


In [35]:
x_test_1, y_test_1 = [], []
x_test_7, y_test_7 = [], []
x_test_15, y_test_15 = [], []
x_test_30, y_test_30 = [], []

In [36]:
for i in range(lookback_window, len(kospi_test)):
    x_test_1.append(kospi_test[i - lookback_window:i])
    y_test_1.append(kospi_test[i] - kospi_test[i-1])

for i in range(lookback_window, len(kospi_test)-6):
    x_test_7.append(kospi_test[i - lookback_window:i])
    y_test_7.append(kospi_test[i+6] - kospi_test[i-1])

for i in range(lookback_window, len(kospi_test)-14):
    x_test_15.append(kospi_test[i - lookback_window:i])
    y_test_15.append(kospi_test[i+14] - kospi_test[i-1])

for i in range(lookback_window, len(kospi_test)-29):
    x_test_30.append(kospi_test[i - lookback_window:i])
    y_test_30.append(kospi_test[i+29] - kospi_test[i-1])

In [37]:
for index in range(len(y_test_1)):
    if y_test_1[index]>= -0.5 and y_test_1[index] < 0.5:
        y_test_1[index] = 'neutral'
    elif y_test_1[index] >= 0.5:
        y_test_1[index] = 'increase'
    else:
        y_test_1[index] = 'decrease'
        
for index in range(len(y_test_7)):
    if y_test_7[index]>= -0.5 and y_test_7[index] < 0.5:
        y_test_7[index] = 'neutral'
    elif y_test_7[index] >= 0.5:
        y_test_7[index] = 'increase'
    else:
        y_test_7[index] = 'decrease'

for index in range(len(y_test_15)):
    if y_test_15[index]>= -0.5 and y_test_15[index] < 0.5:
        y_test_15[index] = 'neutral'
    elif y_test_15[index] >= 0.5:
        y_test_15[index] = 'increase'
    else:
        y_test_15[index] = 'decrease'

for index in range(len(y_test_30)):
    if y_test_30[index]>= -0.5 and y_test_30[index] < 0.5:
        y_test_30[index] = 'neutral'
    elif y_test_30[index] >= 0.5:
        y_test_30[index] = 'increase'
    else:
        y_test_30[index] = 'decrease'

In [38]:
x_test_1 = np.array(x_test_1)
y_test_1 = np.array(y_test_1)
x_test_1 = x_test_1.reshape((1186, -1, 1))
#y_test_1 = y_test_1.reshape((-1, 1))

x_test_7 = np.array(x_test_7)
y_test_7 = np.array(y_test_7)
x_test_7 = x_test_7.reshape((1180, -1, 1))
#y_test_7 = y_test_7.reshape((-1, 1))

x_test_15 = np.array(x_test_15)
y_test_15 = np.array(y_test_15)
x_test_15 = x_test_15.reshape((1172, -1, 1))
#y_test_15 = y_test_15.reshape((-1, 1))

x_test_30 = np.array(x_test_30)
y_test_30 = np.array(y_test_30)
x_test_30 = x_test_30.reshape((1157, -1, 1))
#y_test_30 = y_test_30.reshape((-1, 1))

In [39]:
print(x_test_1.shape, x_test_7.shape, x_test_15.shape, x_test_30.shape)
print(y_test_1.shape, y_test_7.shape, y_test_15.shape, y_test_30.shape)

(1186, 60, 1) (1180, 60, 1) (1172, 60, 1) (1157, 60, 1)
(1186,) (1180,) (1172,) (1157,)


In [41]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
encoder = LabelEncoder()
encoder.fit(y_train_1)

y_train_1_e = to_categorical(encoder.transform(y_train_1))
y_train_7_e = to_categorical(encoder.transform(y_train_7))
y_train_15_e = to_categorical(encoder.transform(y_train_15))
y_train_30_e = to_categorical(encoder.transform(y_train_30))

y_test_1_e = to_categorical(encoder.transform(y_test_1))
y_test_7_e = to_categorical(encoder.transform(y_test_7))
y_test_15_e = to_categorical(encoder.transform(y_test_15))
y_test_30_e = to_categorical(encoder.transform(y_test_30))

In [42]:
model_1 = Sequential([
    TCN(input_shape=(lookback_window, 1),
        kernel_size=2,
        use_skip_connections=False,
        use_batch_norm=False,
        use_weight_norm=False,
        use_layer_norm=False
        ),
    Dense(3, activation='softmax')
])

model_1.summary()
model_1.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
####
model_7 = Sequential([
    TCN(input_shape=(lookback_window, 1),
        kernel_size=2,
        use_skip_connections=False,
        use_batch_norm=False,
        use_weight_norm=False,
        use_layer_norm=False
        ),
    Dense(3, activation='softmax')
])

model_7.summary()
model_7.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
####
model_15 = Sequential([
    TCN(input_shape=(lookback_window, 1),
        kernel_size=2,
        use_skip_connections=False,
        use_batch_norm=False,
        use_weight_norm=False,
        use_layer_norm=False
        ),
    Dense(3, activation='softmax')
])

model_15.summary()
model_15.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
####
model_30 = Sequential([
    TCN(input_shape=(lookback_window, 1),
        kernel_size=2,
        use_skip_connections=False,
        use_batch_norm=False,
        use_weight_norm=False,
        use_layer_norm=False
        ),
    Dense(3, activation='softmax')
])

model_30.summary()
model_30.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

2023-01-25 03:03:38.754817: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/compat/lib:/usr/local/cuda/extras/CUPTI/lib64:/usr/local/cuda/lib:/usr/local/cuda/lib64:/usr/local/nvidia/lib64:/usr/include/x86_64-linux-gnu
2023-01-25 03:03:38.754909: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-01-25 03:03:38.754961: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:163] no NVIDIA GPU device is present: /dev/nvidia0 does not exist
2023-01-25 03:03:38.755779: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in othe

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 tcn (TCN)                   (None, 64)                91136     
                                                                 
 dense (Dense)               (None, 3)                 195       
                                                                 
Total params: 91,331
Trainable params: 91,331
Non-trainable params: 0
_________________________________________________________________
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 tcn_1 (TCN)                 (None, 64)                91136     
                                                                 
 dense_1 (Dense)             (None, 3)                 195       
                                                                 
Total params: 91,331
Trainable

In [43]:
print('Train...')
model_1.fit(x_train_1, y_train_1_e, epochs=100, verbose=2)
p_test_1 = model_1.predict(x_test_1)

print('Train...')
model_7.fit(x_train_7, y_train_7_e, epochs=100, verbose=2)
p_test_7 = model_7.predict(x_test_7)

print('Train...')
model_15.fit(x_train_15, y_train_15_e, epochs=100, verbose=2)
p_test_15 = model_15.predict(x_test_15)

print('Train...')
model_30.fit(x_train_30, y_train_30_e, epochs=100, verbose=2)
p_test_30 = model_30.predict(x_test_30)

Train...
Epoch 1/100
161/161 - 6s - loss: 1082.1036 - accuracy: 0.4726 - 6s/epoch - 36ms/step
Epoch 2/100
161/161 - 3s - loss: 252.8105 - accuracy: 0.4732 - 3s/epoch - 19ms/step
Epoch 3/100
161/161 - 3s - loss: 197.5741 - accuracy: 0.4681 - 3s/epoch - 19ms/step
Epoch 4/100
161/161 - 3s - loss: 152.8845 - accuracy: 0.4683 - 3s/epoch - 19ms/step
Epoch 5/100
161/161 - 3s - loss: 123.6322 - accuracy: 0.4761 - 3s/epoch - 19ms/step
Epoch 6/100
161/161 - 3s - loss: 143.0998 - accuracy: 0.4925 - 3s/epoch - 19ms/step
Epoch 7/100
161/161 - 3s - loss: 92.1990 - accuracy: 0.4800 - 3s/epoch - 19ms/step
Epoch 8/100
161/161 - 3s - loss: 87.0174 - accuracy: 0.4777 - 3s/epoch - 19ms/step
Epoch 9/100
161/161 - 3s - loss: 68.8598 - accuracy: 0.4611 - 3s/epoch - 18ms/step
Epoch 10/100
161/161 - 3s - loss: 71.0317 - accuracy: 0.4654 - 3s/epoch - 19ms/step
Epoch 11/100
161/161 - 3s - loss: 76.1801 - accuracy: 0.4752 - 3s/epoch - 19ms/step
Epoch 12/100
161/161 - 3s - loss: 77.5754 - accuracy: 0.4697 - 3s/epo

Epoch 99/100
161/161 - 3s - loss: 13.8462 - accuracy: 0.4722 - 3s/epoch - 19ms/step
Epoch 100/100
161/161 - 3s - loss: 10.0431 - accuracy: 0.4681 - 3s/epoch - 19ms/step
38/38 [==============================] - 1s 7ms/step
Train...
Epoch 1/100
161/161 - 6s - loss: 1099.6788 - accuracy: 0.4930 - 6s/epoch - 35ms/step
Epoch 2/100
161/161 - 3s - loss: 262.5837 - accuracy: 0.4922 - 3s/epoch - 19ms/step
Epoch 3/100
161/161 - 3s - loss: 173.9264 - accuracy: 0.5025 - 3s/epoch - 19ms/step
Epoch 4/100
161/161 - 3s - loss: 161.2151 - accuracy: 0.5099 - 3s/epoch - 19ms/step
Epoch 5/100
161/161 - 3s - loss: 108.3744 - accuracy: 0.4984 - 3s/epoch - 19ms/step
Epoch 6/100
161/161 - 3s - loss: 105.7679 - accuracy: 0.4990 - 3s/epoch - 19ms/step
Epoch 7/100
161/161 - 3s - loss: 82.9257 - accuracy: 0.5090 - 3s/epoch - 19ms/step
Epoch 8/100
161/161 - 3s - loss: 78.3811 - accuracy: 0.4957 - 3s/epoch - 19ms/step
Epoch 9/100
161/161 - 3s - loss: 71.0074 - accuracy: 0.5016 - 3s/epoch - 19ms/step
Epoch 10/100
16

Epoch 97/100
161/161 - 3s - loss: 8.9297 - accuracy: 0.4916 - 3s/epoch - 19ms/step
Epoch 98/100
161/161 - 3s - loss: 7.1152 - accuracy: 0.4967 - 3s/epoch - 19ms/step
Epoch 99/100
161/161 - 3s - loss: 7.4768 - accuracy: 0.5000 - 3s/epoch - 19ms/step
Epoch 100/100
161/161 - 3s - loss: 8.7963 - accuracy: 0.5014 - 3s/epoch - 19ms/step
37/37 [==============================] - 0s 6ms/step
Train...
Epoch 1/100
160/160 - 6s - loss: 1552.8459 - accuracy: 0.5188 - 6s/epoch - 37ms/step
Epoch 2/100
160/160 - 3s - loss: 407.6935 - accuracy: 0.4941 - 3s/epoch - 19ms/step
Epoch 3/100
160/160 - 3s - loss: 204.0381 - accuracy: 0.5190 - 3s/epoch - 19ms/step
Epoch 4/100
160/160 - 3s - loss: 242.4633 - accuracy: 0.5125 - 3s/epoch - 19ms/step
Epoch 5/100
160/160 - 3s - loss: 151.2382 - accuracy: 0.4957 - 3s/epoch - 20ms/step
Epoch 6/100
160/160 - 3s - loss: 70.5611 - accuracy: 0.5094 - 3s/epoch - 19ms/step
Epoch 7/100
160/160 - 3s - loss: 87.7695 - accuracy: 0.5123 - 3s/epoch - 19ms/step
Epoch 8/100
160/16

Epoch 94/100
160/160 - 3s - loss: 8.1823 - accuracy: 0.4861 - 3s/epoch - 19ms/step
Epoch 95/100
160/160 - 3s - loss: 6.7995 - accuracy: 0.5117 - 3s/epoch - 19ms/step
Epoch 96/100
160/160 - 3s - loss: 11.2597 - accuracy: 0.5066 - 3s/epoch - 20ms/step
Epoch 97/100
160/160 - 3s - loss: 11.4574 - accuracy: 0.5006 - 3s/epoch - 19ms/step
Epoch 98/100
160/160 - 3s - loss: 10.6992 - accuracy: 0.5006 - 3s/epoch - 19ms/step
Epoch 99/100
160/160 - 3s - loss: 12.6951 - accuracy: 0.5074 - 3s/epoch - 19ms/step
Epoch 100/100
160/160 - 3s - loss: 9.0685 - accuracy: 0.5098 - 3s/epoch - 19ms/step
37/37 [==============================] - 0s 6ms/step
Train...
Epoch 1/100
160/160 - 6s - loss: 840.3288 - accuracy: 0.5142 - 6s/epoch - 37ms/step
Epoch 2/100
160/160 - 3s - loss: 372.5275 - accuracy: 0.5093 - 3s/epoch - 19ms/step
Epoch 3/100
160/160 - 3s - loss: 193.1499 - accuracy: 0.5150 - 3s/epoch - 19ms/step
Epoch 4/100
160/160 - 3s - loss: 144.9919 - accuracy: 0.5181 - 3s/epoch - 19ms/step
Epoch 5/100
160/

Epoch 92/100
160/160 - 3s - loss: 8.9416 - accuracy: 0.5175 - 3s/epoch - 19ms/step
Epoch 93/100
160/160 - 3s - loss: 9.8916 - accuracy: 0.5058 - 3s/epoch - 19ms/step
Epoch 94/100
160/160 - 3s - loss: 9.4130 - accuracy: 0.5179 - 3s/epoch - 19ms/step
Epoch 95/100
160/160 - 3s - loss: 9.5585 - accuracy: 0.5093 - 3s/epoch - 19ms/step
Epoch 96/100
160/160 - 3s - loss: 13.0052 - accuracy: 0.5107 - 3s/epoch - 20ms/step
Epoch 97/100
160/160 - 3s - loss: 8.4812 - accuracy: 0.5128 - 3s/epoch - 19ms/step
Epoch 98/100
160/160 - 3s - loss: 10.6291 - accuracy: 0.5097 - 3s/epoch - 20ms/step
Epoch 99/100
160/160 - 3s - loss: 8.6501 - accuracy: 0.5126 - 3s/epoch - 20ms/step
Epoch 100/100
160/160 - 3s - loss: 8.9667 - accuracy: 0.5117 - 3s/epoch - 19ms/step
37/37 [==============================] - 1s 7ms/step


In [45]:
model_1.save('kospi_1day_classification_v2.h5')
model_7.save('kospi_7days_classification_v2.h5')
model_15.save('kospi_15days_classification_v2.h5')
model_30.save('kospi_30days_classification_v2.h5')

In [50]:
print('\n Accuracy: %.4f' % (model_1.evaluate(x_test_1, y_test_1_e)[1]))
print('\n Accuracy: %.4f' % (model_7.evaluate(x_test_7, y_test_7_e)[1]))
print('\n Accuracy: %.4f' % (model_15.evaluate(x_test_15, y_test_15_e)[1]))
print('\n Accuracy: %.4f' % (model_30.evaluate(x_test_30, y_test_30_e)[1]))

38/38 [==============================] - 0s 10ms/step - loss: 37.9323 - accuracy: 0.5261

 Accuracy: 0.5261
37/37 [==============================] - 1s 10ms/step - loss: 6.3236 - accuracy: 0.4763

 Accuracy: 0.4763
37/37 [==============================] - 1s 10ms/step - loss: 38.2471 - accuracy: 0.5000

 Accuracy: 0.5000
37/37 [==============================] - 1s 9ms/step - loss: 5.9613 - accuracy: 0.4978

 Accuracy: 0.4978
